In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import time
import networkx as nx
import matplotlib as mpl
#install pytrends
#!pip install pytrends
from pytrends.request import TrendReq

In [ ]:
# scale data based on which year and country we're interested in
# Idea: Correct the data by scaling it here and write a new csv-file with this new data, before using it further.

def get_historical_dataframe(keyword, timeframe, geo=''):
    kw_list = [keyword]
    pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25), retries=2, backoff_factor=0.1, requests_args={'verify':False})
    pytrend.build_payload(kw_list, timeframe=timeframe, geo=geo)
    
    # next we make the dataframe for this word over the period
    BP = pytrend.interest_over_time()
    if BP.empty:
        BP = pd.DataFrame([1,1],columns=[keyword])

    return BP

def scale_data(df_nor,years,keyword,geo=''):
    '''
    df_nor: dataframe of one country
    years: years we want to stitch together (can get this from df)
    geo: code name of country
    column: which column we want to scale. 2 for first keyword
    returns a dataframe where the specified column is scaled appropriately
    '''
      
    df_nor.reset_index(inplace=True,drop=True)
    
    # get dates where we patch
    patch_dates_from = []
    patch_dates_to = []
    for year in years:
        patch_dates_from.append(f"{year}-06-30")
        patch_dates_to.append(f"{year}-07-01")
        patch_dates_from.append(f"{year}-12-31")
        patch_dates_to.append(f"{year+1}-01-01")
    patch_dates_from.pop()
    patch_dates_to.pop()
    
    for i in range(len(patch_dates_from)):
        # prepare data
        
        i_date_from = df_nor.index[df_nor["date"]==patch_dates_from[i]].tolist()[0]
        i_date_to = df_nor.index[df_nor["date"]==patch_dates_to[i]].tolist()[0]
        date_from = df_nor["date"][i_date_from]
        date_to = df_nor["date"][i_date_to]
        timeframe = date_from + " " + date_to
        #print(timeframe)
        P = [df_nor[keyword][i_date_from],df_nor[keyword][i_date_to]]
        #print(P)
        
        BP = get_historical_dataframe(keyword,timeframe,geo)
        print(BP)
        BP_vals = [BP[keyword][0],BP[keyword][1]]
        
        # actually do the scaling
        s = BP_vals[1]/BP_vals[0]*P[0]/P[1]
        #print(df[keyword][0:])
        df[keyword][i_date_from:] = df[keyword][i_date_from:]*s
        
    df[keyword] = 100*df[keyword]/df[keyword].max()
    
    return df
        

'''
filename = "bigboy.csv"
df = pd.read_csv(filename)


YEARS = [2010]#, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df_nor = df.loc[df['country'] == "Norway"]
new_df = scale_data(df_nor,YEARS,"Facebook")
print(len(df.loc[df['country'] == "Norway"]))
df.loc[df['country'] == "Norway"] = new_df
print(new_df)
print(len(df.loc[df['country'] == "Norway"]))
# df.loc[df['country'] == "Norway"] = new_df => overwrites only for Norway. 
# Can loop over each country and search term to make new csv file
'''

In [ ]:
def _load_unavailable_countries(filename: str):
    unavailable_countries = []
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            unavailable_countries.append(line)
    return unavailable_countries

def load_countries(filename: str, ignore: str):
    unavailable_countries = _load_unavailable_countries(ignore)
    valid_countries = []
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            line_list = line.split(",")
            code = line_list[0]
            country = line_list[1]
            if country not in unavailable_countries:
                valid_countries.append((code, country))
    return valid_countries

In [ ]:
filename = "bigboy.csv"
df = pd.read_csv(filename)
df.reset_index(drop=True)
to_filename = "scaled_data.csv"

YEARS = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
search_term = "Facebook"
country = "Norway"

country_df = df.loc[df["country"] == country]

scale_data(country_df,YEARS,search_term,"NO")

# before inserting the changed values into the original dataframe, make sure to match indexes
country_df.index = df.loc[df['country'] == country].index
# insert into dataframe
df.loc[df["country"] == country] = country_df

search_term = "Facebook"
country = "Sweden"

country_df = df.loc[df["country"] == country]

scale_data(country_df,YEARS,search_term,"SE")

# before inserting the changed values into the original dataframe, make sure to match indexes
country_df.index = df.loc[df['country'] == country].index
# insert into dataframe
df.loc[df["country"] == country] = country_df

df.to_csv(to_filename)

In [ ]:
filename = "bigboy.csv"
to_filename = "scaled_data.csv"

valid_countries = load_countries("docs/countries.txt","docs/ignore.txt")
countries = df["country"].unique()
search_terms = df.columns[2:]

YEARS = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df = pd.read_csv(filename)
i = 0
for search_term in search_terms:
    for country in countries:
        geo = ""
        for geocode,check_country in valid_countries:
            if country == check_country:
                geo = geocode
                print(country,geo)
        # patch data using search_term,country,geocode
        '''
        How: 
        make new df containing only the data for country
        scale this data
        '''
        print(i)
        country_df = df.loc[df["country"] == country]

        scale_data(country_df,YEARS,search_term,geo)

        # before inserting the changed values into the original dataframe, make sure to match indexes
        country_df.index = df.loc[df['country'] == country].index
        # insert into dataframe
        df.loc[df["country"] == country] = country_df
    pf.to_csv(f"scale/try_{i}_{to_filename}")
    i += 1



'''
Rate Limit is not publicly known, let me know if you have a consistent estimate
* One user reports that 1,400 sequential requests of a 4 hours timeframe got them to the limit. (Replicated on 2 networks)
* It has been tested, and 60 seconds of sleep between requests (successful or not) appears to be the correct amount once you reach the limit.
time.sleep(60)
'''

In [ ]:
df

In [ ]:
def get_corr_matrix(filename,category,glob=False):
    '''
    filename: Trends/filename (remember .csv). ex: "db_0.csv"
    category: ex "Facebook", "YouTube"
    returns list of all countries involved (all names of columns/rows) and correlation matrix 
    '''
    df = pd.read_csv(f"{filename}")
    df["date"]= pd.to_datetime(df["date"])
    df.dropna(inplace=True)
    
    # Keep country and Facebook
    df = df[["country", category]]
    
    # Get a list of countries
    countries = df["country"].unique()
    
    # Create a new DataFrame 
    new_df = pd.DataFrame()
    for country in countries:
        tmp = df[df["country"] == country]#
        tmp = tmp.drop(columns=["country"])
        tmp = tmp.rename(columns={category: country})
        new_df = pd.concat([new_df.reset_index(drop=True), tmp.reset_index(drop=True)], axis=1)
    
    if glob:
        # Remove Global, and insert it at the front
        new_df["Global"] = new_df.sum(axis=1) / new_df.shape[1]
        GLOB = new_df.pop('Global')
        new_df.insert(0, "Global", GLOB)
    
    new_df = new_df.sort_index(axis=1)
        
    CORRELATION_MATRIX = new_df.corr()
    
    return CORRELATION_MATRIX.columns, CORRELATION_MATRIX
    
cnt,mtr = get_corr_matrix("bigboy.csv","Facebook")

In [ ]:
# get categories for each file
def get_categories(filename):
    '''
    filename: filename (.csv)
    returns categories as a 2D-list where each
    inner list refers to a filename
    '''    
    df = pd.read_csv(f"{filename}")
    df["date"]= pd.to_datetime(df["date"])
    df.dropna(inplace=True)
    categories = df.columns[2:]
    return categories

In [ ]:
# get corr matrix for bigboy.csv and scaled_data.csv

filename = "bigboy.csv"
category = "Facebook"
cntr, corr_matrix_1 = get_corr_matrix(filename,category)

filename = "scaled_data.csv"
category = "Facebook"
cntr, corr_matrix_2 = get_corr_matrix(filename,category)

In [ ]:
print(corr_matrix_1["Norway"]["Sweden"],corr_matrix_2["Norway"]["Sweden"],corr_matrix_1["Norway"]["Sweden"]/corr_matrix_2["Norway"]["Sweden"])

In [ ]:
# get corr matrix
# prepare to get corr matrices
filename = "bigboy.csv"
categories = get_categories(filename)
#print(filename,categories)

# get it for each category/term
corr_matrices = []
for cat in categories:
    countries, corr_matrix = get_corr_matrix(filename,cat)
    corr_matrices.append([cat,countries,corr_matrix])
corr_matrices

In [ ]:
avg_corr_matrix = corr_matrices[0][2].copy()

for col in avg_corr_matrix.columns:
    avg_corr_matrix[col].values[:] = 0

    
for i in range(45,50):
    i_matrix = corr_matrices[i][2].copy().fillna(1)
    avg_corr_matrix += i_matrix


avg_corr_matrix /= 5
print(avg_corr_matrix)

In [ ]:
def convert_matrix(corr_matrix,treshold):
    '''
    corr_matrix: correlation matrix (dataframe)
    treshold: value between 0 and 1
    converts matrix to a matrix where all values > treshold gets replaced with 1 and the rest 0
    '''
    matrix = corr_matrix.copy()
    matrix[matrix>=treshold] = 1
    matrix[matrix<treshold] = 0
    
    return matrix

def make_groups_from_matrix(corr_matrix, treshold, used_countries):
    '''
    corr_matrix: correlation matrix (dataframe)
    treshold: value between 0 and 1
    used_countries: countries you don't want to be grouped up. Useful if we already have a group
    returns list of lists, where each list is a group
    '''
    matrix = convert_matrix(corr_matrix,treshold)
    #print(corr_matrix,treshold,used_countries)
    groups = []
    countries = matrix.columns
    for c in countries:
        if c not in used_countries:
            # if country not visited before, make a new group and make sure all correlated countries are added
            group = [c]
            used_countries.append(c)             
            
            for ct in group:
                # make list of countries that ct correlates with
                # append all countries that has not been visited before to group and used_countries
                ct_corr = matrix[ct][matrix[ct]==1].index.values
                for ctr in ct_corr:
                    if ctr not in used_countries:
                        used_countries.append(ctr)
                        group.append(ctr)
            
            
            groups.append(group)
    return groups

In [ ]:
# Method 1: Is for sure making larger groups. Might look nice, but doesn't necessarily say that much
# Idea: Make groups with high treshold, then of the remaining countries, use lower treshold, until reach 0 as treshold

def groupifier(corr_matrix,start_treshold=0.9,skip_treshold=0.1):
    '''
    returns groups in a list, and groups on the form: [[treshold,[groups]]xG] (G groups)
    '''
    groups = []
    groups_treshold = []
    used_countries = []
    steps = int(start_treshold/skip_treshold+1)
    for treshold in np.linspace(start_treshold,0,steps):
        gr = make_groups_from_matrix(corr_matrix, treshold, used_countries[:])
        # copy group
        gr_copy = gr[:]
        for g in gr_copy:
            #print(g)
            if len(g)==1:
                gr.remove(g)
            else:
                used_countries+=g
                groups.append(g)
        if len(gr):
            groups_treshold.append([round(treshold,1),gr])

    return groups,groups_treshold
        
comat = corr_matrices[0][2]
#print(comat)
groups, groups_treshold = groupifier(comat,0.9,0.1)
print(groups)

In [ ]:
# Method 2: Doesn't necessarily give big groups, but makes sure groups are well defined
# Meaning: All countries in each group correlates at least treshold to all other countries in that group

def groupifier2(corr_matrix,treshold):
    '''
    corr_matrix: correlation matrix
    treshold: number between 0 and 1
    returns groups where all countries in each group correlate
        at least treshold with each other country in the group
    '''
    # make a list containing a list with each country
    groups = [corr_matrix.columns.values.tolist()]
    # idea: split group into sub-groups until each sub-group fullfills criteria
    #print(len(groups[0]))
    while True:
        new_groups = []
        for group in groups:
            # find countries that correlate least
            country1 = ""
            country2 = ""
            least_treshold = 1
            for c1 in group:
                for c2 in group:
                    if corr_matrix[c1][c2] < least_treshold:
                        country1 = c1
                        country2 = c2
                        least_treshold = corr_matrix[c1][c2]
            if least_treshold < treshold:
                break_loop = False
                g1 = [country1]
                g2 = [country2]
                for country in group:
                    if country != country1 and country != country2:
                        if corr_matrix[country1][country] > corr_matrix[country2][country]:
                            g1.append(country)
                        else:
                            g2.append(country)
                new_groups.append(g1)
                new_groups.append(g2)
            else:
                new_groups.append(group)
                       
        # if no changes are done, break loop. if not, 
        if new_groups==groups:
            break
        else:
            groups = new_groups
    
    return groups


comat = corr_matrices[46][2]
groups_treshold = groupifier2(comat,0.7)
print(groups_treshold)
print(len(groups_treshold))

In [ ]:
gr1 = make_groups_from_matrix(avg_corr_matrix, 0.8, [])

gr2, gr2_tresh = groupifier(avg_corr_matrix)

gr3 = groupifier2(avg_corr_matrix,0.8)

print(len(gr1),gr1,"\n\n\n",len(gr2),gr2_tresh,"\n\n\n",len(gr3),gr3)

In [ ]:
for i in range(len(gr2_tresh)):
    for j in range(len(gr2_tresh[i][1])):
        df = pd.DataFrame(gr2_tresh[i][1][j])
        df.columns = [gr2_tresh[i][0]]
        df.to_csv(f"groups/treshold_{gr2_tresh[i][0]}_group_{j+1}.csv",index=False)


In [ ]:
print(gr1)
for i in range(len(gr1)):
    df = pd.DataFrame(gr1[i])
    df.columns = [f"group {i+1}"]
    print(df)
    df.to_csv(f"groups2/group_{i+1}.csv",index=False)

In [ ]:
for i in range(len(corr_matrices)):
    print(corr_matrices[i][0],corr_matrices[i][2]["Norway"]["Sweden"])

In [ ]:
avg_corr_matrix2 = corr_matrices[0][2].copy()

for col in avg_corr_matrix.columns:
    avg_corr_matrix2[col].values[:] = 0

    
for i in range(45,50):
    i_matrix = corr_matrices[i][2].copy().fillna(1)
    avg_corr_matrix2 += i_matrix


avg_corr_matrix2 /= 5
print(avg_corr_matrix2,"\n\n\n",avg_corr_matrix2["Norway"]["Sweden"])

In [ ]:
gr1 = make_groups_from_matrix(avg_corr_matrix2, 0.5, [])

gr2, gr2_tresh = groupifier(avg_corr_matrix2)

gr3 = groupifier2(avg_corr_matrix2,0.5)

print(len(gr1),gr1,"\n\n\n",len(gr2),gr2_tresh,"\n\n\n",len(gr3),gr3)

In [ ]:
avg_corr_matrix = corr_matrices[45][2]

G = nx.Graph()
for i in range(len(avg_corr_matrix)):
    for j in range(i,len(avg_corr_matrix)):
        if j != i:
            G.add_edge(avg_corr_matrix.columns[i], avg_corr_matrix[avg_corr_matrix.columns[i]].index[j], weight = avg_corr_matrix[avg_corr_matrix.columns[i]][avg_corr_matrix[avg_corr_matrix.columns[i]].index[j]])

df = pd.DataFrame()

In [ ]:
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams['figure.dpi'] = 200

In [ ]:
edges = G.edges()
weights = [G[u][v]['weight'] for u,v in edges]


nc = nx.draw_networkx(G, with_labels=True, node_color='lightgreen', edge_color='#aaaaaa', node_size=350, width=weights, font_size=8, font_color='black', font_weight='bold')
